# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f3291c2f8a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Trademarks across the world") 

Consider the following exploratory information need:

> You are investigating different kinds of trademarks

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wd:Q167270`  | trademark     | node      |
| `wd:Q14091`   | iMac          | node      |
| `wd:Q312`     | Apple Inc.    | node      |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q167270  . 
```

is the BGP to retrieve all **entities that are trademarks**

## Workload Goals

1. Identify the BGP for the company or person related to a given trademark

2. Identify the BGP to retrieve types or categories for a given trademark

3. What companies have the largest number of trademarks? 

4. What are the types of trademarks, how many trademarks exist for each type?

5. Analyze the number of trademarks across types, companies, and countries
 
   5.1 How many U.S. companies hold trademarks? In which sectors?
   
   5.2 How many people hold or are connected to trademarks? In which role?
   
   5.3 In which sector there is the highest number of trademarks?   


In [1]:
# start your workflow here

In [6]:
queryString = """
SELECT COUNT(?p)
WHERE { 

?p wdt:P31/wdt:P279* wd:Q167270  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '44700')]


1

## Task 1 -Identify the BGP for the company or person related to a given trademark

#### Let's inspect all properties of trademark

In [16]:
queryString = """
SELECT ?p ?pname (COUNT(?tm) as ?number) WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    ?p ?obj.

?p <http://schema.org/name> ?pname.
} 
GROUP BY ?p ?pname
ORDER BY DESC (?number)
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('number', '61037')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('number', '28077')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception'), ('number', '24438')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pname', 'official website'), ('number', '21505')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('number', '19129')]
[('p', 'http://www.wikidata.org/prop/direct/P966'), ('pname', 'MusicBrainz label ID'), ('number', '12972')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('number', '12948')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID'), ('number', '12310')]
[('p', 'http://www.wikidata.org/prop/direct/P1955'), ('pname', 'Discogs label ID'), ('number', '8263')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pname', 'F

200

In [17]:
queryString = """
SELECT ?p ?pname (COUNT(?tm) as ?number) WHERE {

?sub ?p ?tm.
?tm wdt:P31/wdt:P279* wd:Q167270.
   

?p <http://schema.org/name> ?pname.
} 
GROUP BY ?p ?pname
ORDER BY DESC (?number)
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('pname', 'record label'), ('number', '669602')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('pname', 'publisher'), ('number', '12236')]
[('p', 'http://www.wikidata.org/prop/direct/P1716'), ('pname', 'brand'), ('number', '10096')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pname', 'distributed by'), ('number', '6918')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('number', '6092')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pname', 'manufacturer'), ('number', '4673')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator'), ('number', '4487')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics'), ('number', '4087')]
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('pname', 'production company'), ('number', '3889')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('number', '

200

#### I inspect for Example "Apple Inc" in search of the Company class

In [4]:
queryString = """
SELECT ?obj ?objname WHERE {

wd:Q312 wdt:P31 ?obj.

?obj <http://schema.org/name> ?objname.
} 

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q431289'), ('objname', 'brand')]
[('obj', 'http://www.wikidata.org/entity/Q4830453'), ('objname', 'business')]
[('obj', 'http://www.wikidata.org/entity/Q6881511'), ('objname', 'enterprise')]
[('obj', 'http://www.wikidata.org/entity/Q891723'), ('objname', 'public company')]
[('obj', 'http://www.wikidata.org/entity/Q167270'), ('objname', 'trademark')]
[('obj', 'http://www.wikidata.org/entity/Q167037'), ('objname', 'corporation')]


6

#### I notice that a company can be an instance also for his corresponding brand

In [7]:
queryString = """
SELECT DISTINCT ?tmname  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P31 wd:Q891723.

?tm <http://schema.org/name> ?tmname.
#?c <http://schema.org/name> ?company.
} 

"""

print("Results")
run_query(queryString)

Results
[('tmname', 'FNC Entertainment')]
[('tmname', 'YG Entertainment')]
[('tmname', 'Orange Sky Golden Harvest')]
[('tmname', 'Foot Locker')]
[('tmname', 'Huayi Brothers')]
[('tmname', 'CJ E&M')]
[('tmname', 'Carmike Cinemas')]
[('tmname', 'AMC Theatres')]
[('tmname', 'Stein Mart')]
[('tmname', 'Cineworld')]
[('tmname', 'SM Entertainment')]
[('tmname', 'JYP Entertainment')]
[('tmname', 'Kmart')]
[('tmname', 'Amuse')]
[('tmname', 'SM Culture & Contents')]
[('tmname', 'Cineplex Entertainment')]
[('tmname', 'INOX Leisure Limited')]
[('tmname', 'Cinemark Theatres')]
[('tmname', 'PVR Cinemas')]
[('tmname', 'Takashimaya')]
[('tmname', 'Finish Line, Inc.')]
[('tmname', 'China Film Stellar')]
[('tmname', 'Parco')]
[('tmname', 'Wanda Cinemas')]
[('tmname', 'Kintetsu Department Store')]
[('tmname', 'Matahari Department Store')]
[('tmname', 'Designer Brands')]
[('tmname', 'Ramayana Lestari Sentosa')]
[('tmname', 'Payless ShoeSource')]
[('tmname', 'Shoe Carnival')]
[('tmname', 'Berliner Kindl')

253

#### I try with "owned by"(wdt:P127)

In [5]:
queryString = """
SELECT DISTINCT ?objname WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?obj.

?obj <http://schema.org/name> ?objname.
} 
ORDER BY ASC (?objname)

"""

print("Results")
run_query(queryString)

Results
[('objname', '21st Century Fox')]
[('objname', '3M')]
[('objname', '3i')]
[('objname', '4ALL')]
[('objname', "A.C.A. Atelier d'Etude de constructions automobiles S.à r.l.")]
[('objname', 'A.G. Barr')]
[('objname', 'A.S. Watson Group')]
[('objname', 'AB InBev Ukraine')]
[('objname', 'AB Wilh. Becker')]
[('objname', 'ABC Records')]
[('objname', 'ABC Records, Inc.')]
[('objname', 'ABC Supermarkets AS')]
[('objname', 'ABC-Paramount Records, Inc.')]
[('objname', 'AEA Investors')]
[('objname', 'AGCO')]
[('objname', 'ALBATROS nakladatelství, a.s.')]
[('objname', 'AMC Theatres')]
[('objname', 'ARGO SpA')]
[('objname', 'AT&T')]
[('objname', 'ATI Technologies')]
[('objname', 'AUSA')]
[('objname', 'AWA Studios')]
[('objname', 'Abalena Capital Limited')]
[('objname', 'Abercrombie & Fitch')]
[('objname', 'Access Industries')]
[('objname', 'Accor')]
[('objname', 'Acer Inc.')]
[('objname', 'Acer Studios')]
[('objname', 'Achel Abbey')]
[('objname', 'Adam Foods')]
[('objname', 'Adecoagro')]
[('

100

#### Now i combine with "founded by" (wdt:P112) to get also the founders of the company that hold the trademark

In [ ]:
queryString = """
SELECT DISTINCT ?tmname ?objname (GROUP_CONCAT(?foundername; separator=", ") as ?founders) WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270.
OPTIONAL{ ?tm wdt:P127 ?obj. ?obj <http://schema.org/name> ?objname.}
OPTIONAL{ ?tm wdt:P112 ?founder. ?founder <http://schema.org/name> ?foundername.}
?tm <http://schema.org/name> ?tmname.
} 
ORDER BY ASC (?tmname)

"""

print("Results")
run_query(queryString)

#### In conclusion it seems that world's most important tradermarks are "stored" within the company entity (like Apple, Mcdonal's ecc). For other trademarks i was able to obtain the name of some company shareholders along with founders

#### I run the complete query for "Apple INc" (wd:Q312)

In [6]:
queryString = """
SELECT DISTINCT  (GROUP_CONCAT(?objname; separator=", ") AS ?shareholders) (GROUP_CONCAT(?foundername; separator=", ") as ?founders) WHERE {

OPTIONAL{ wd:Q312 wdt:P127 ?obj. ?obj <http://schema.org/name> ?objname.}
OPTIONAL{ wd:Q312 wdt:P112 ?founder. ?founder <http://schema.org/name> ?foundername.}
} 
"""
print("Results")
run_query(queryString)

Results
[('shareholders', 'The Vanguard Group, The Vanguard Group, The Vanguard Group, Berkshire Hathaway, Berkshire Hathaway, Berkshire Hathaway, Government Pension Fund Global, Government Pension Fund Global, Government Pension Fund Global, BlackRock, BlackRock, BlackRock'), ('founders', 'Steve Wozniak, Steve Jobs, Ronald Wayne, Steve Wozniak, Ronald Wayne, Steve Jobs, Steve Jobs, Ronald Wayne, Steve Wozniak, Ronald Wayne, Steve Jobs, Steve Wozniak')]


1

## Task 2 - Identify the BGP to retrieve types or categories for a given trademark

#### Let's try with the property "genre" (wdt:P136) found before

In [12]:
queryString = """
SELECT DISTINCT ?tmname ?objname  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P136 ?obj.

?tm <http://schema.org/name> ?tmname.
?obj <http://schema.org/name> ?objname.
} 
ORDER BY ASC (?tmname)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('tmname', '1%'), ('objname', 'rap')]
[('tmname', '1%'), ('objname', 'hip hop')]
[('tmname', '1%'), ('objname', 'hip hop music')]
[('tmname', '1017 Records'), ('objname', 'hip hop music')]
[('tmname', '1111Klubb'), ('objname', 'hip hop music')]
[('tmname', '1111Klubb'), ('objname', 'pop music')]
[('tmname', '1333 Entertainment'), ('objname', 'hip hop music')]
[('tmname', '1333 Entertainment'), ('objname', 'pop music')]
[('tmname', '1333 Entertainment'), ('objname', 'rhythm and blues')]
[('tmname', '1333 Entertainment'), ('objname', 'electronic dance music')]
[('tmname', '13th Planet Records'), ('objname', 'heavy metal')]
[('tmname', '157 Shelter Records'), ('objname', 'house music')]
[('tmname', '187 Strassenbande'), ('objname', 'gangsta rap')]
[('tmname', '20 Buck Spin'), ('objname', 'heavy metal')]
[('tmname', '2101 Records'), ('objname', 'dance music')]
[('tmname', '21st Circuitry'), ('objname', 'industrial rock')]
[('tmname', '2419 Record Label'), ('objname', 'jazz')]
[('t

100

#### Ok no, genre is only about music trademark

#### I try with another property found: "industry"(wdt:P452)

In [13]:
queryString = """
SELECT DISTINCT ?tmname ?objname  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P452 ?obj.

?tm <http://schema.org/name> ?tmname.
?obj <http://schema.org/name> ?objname.
} 
ORDER BY ASC (?tmname)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('tmname', '%Arabica'), ('objname', 'coffee')]
[('tmname', '01055'), ('objname', 'telecommunication')]
[('tmname', '100PROmotion'), ('objname', 'music industry')]
[('tmname', '101 Drogerie'), ('objname', 'drugstore')]
[('tmname', '1333 Entertainment'), ('objname', 'music industry')]
[('tmname', '2-Tone Entertainment'), ('objname', 'music industry')]
[('tmname', '24 Hour Fitness'), ('objname', 'fitness industry')]
[('tmname', '3M'), ('objname', 'conglomerate')]
[('tmname', '4 Asa U Kući'), ('objname', 'music industry')]
[('tmname', '430 West Records'), ('objname', 'music')]
[('tmname', '7-Eleven'), ('objname', 'convenience store')]
[('tmname', '88rising'), ('objname', 'music industry')]
[('tmname', '8D Creative'), ('objname', 'music industry')]
[('tmname', '960 Music Group'), ('objname', 'entertainment')]
[('tmname', '99 Cents Only Stores'), ('objname', 'discount store')]
[('tmname', 'A Pea in the Pod'), ('objname', 'retail')]
[('tmname', 'A&M Octone Records'), ('objname', 'pho

100

#### Found the type of trademark, now for example let's try to print the types of "Apple"

In [14]:
queryString = """
SELECT DISTINCT  ?objname  WHERE {

wd:Q312 wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?obj.

?obj <http://schema.org/name> ?objname.
} 

"""

print("Results")
run_query(queryString)

Results
[('objname', 'consumer electronics')]
[('objname', 'electronics')]
[('objname', 'mobile phone industry')]
[('objname', 'information technology')]
[('objname', 'software industry')]
[('objname', 'digital distribution')]


6

## Task 3 - What companies have the largest number of trademarks?

#### I reuse the query found in task 1 and aggregate by companies

In [18]:
queryString = """
SELECT DISTINCT ?objname (COUNT( DISTINCT ?tm) AS ?number)  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?obj.
    
?obj <http://schema.org/name> ?objname.
?tm <http://schema.org/name> ?tmname.
} 
GROUP BY (?objname)
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('objname', 'Nestlé'), ('number', '68')]
[('objname', 'Covestro'), ('number', '57')]
[('objname', 'Mondelēz International'), ('number', '32')]
[('objname', 'Unilever'), ('number', '32')]
[('objname', 'Procter & Gamble'), ('number', '29')]
[('objname', 'Universal Music Group'), ('number', '29')]
[('objname', 'The Hershey Company'), ('number', '23')]
[('objname', 'PepsiCo'), ('number', '21')]
[('objname', 'Kraft Heinz'), ('number', '21')]
[('objname', 'Warner Music Group'), ('number', '19')]
[('objname', 'Cadbury'), ('number', '18')]
[('objname', 'Sony Music'), ('number', '18')]
[('objname', 'Mars, Incorporated'), ('number', '17')]
[('objname', 'The Coca-Cola Company'), ('number', '16')]
[('objname', "Kellogg's"), ('number', '16')]
[('objname', 'EMI'), ('number', '14')]
[('objname', 'Ferrero SpA'), ('number', '13')]
[('objname', 'Reckitt'), ('number', '12')]
[('objname', 'Johnson & Johnson'), ('number', '12')]
[('objname', 'General Mills'), ('number', '11')]
[('objname', 'Accor'

1589

## Task 4 - What are the types of trademarks, how many trademarks exist for each type?

#### I reuse the query of task 2

In [20]:
queryString = """
SELECT DISTINCT  ?typename (COUNT(?tm) AS ?number)  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?type.

?type <http://schema.org/name> ?typename.
} 
GROUP BY (?typename)
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('typename', 'retail'), ('number', '1652')]
[('typename', 'automotive industry'), ('number', '509')]
[('typename', 'music industry'), ('number', '413')]
[('typename', 'restaurant'), ('number', '372')]
[('typename', 'gastronomy'), ('number', '272')]
[('typename', 'brewing'), ('number', '252')]
[('typename', 'consumer electronics'), ('number', '232')]
[('typename', 'clothing industry'), ('number', '210')]
[('typename', 'fast food'), ('number', '175')]
[('typename', 'entertainment'), ('number', '165')]
[('typename', 'music'), ('number', '154')]
[('typename', 'food industry'), ('number', '120')]
[('typename', 'hospitality industry'), ('number', '119')]
[('typename', 'phonographic industry'), ('number', '94')]
[('typename', 'foodservice'), ('number', '90')]
[('typename', 'video game industry'), ('number', '84')]
[('typename', 'grocery store'), ('number', '80')]
[('typename', 'supermarket'), ('number', '68')]
[('typename', 'fashion'), ('number', '64')]
[('typename', 'food and tobacc

475

## Task 5 - Analyze the number of trademarks across types, companies, and countries

#### I retrieve the country (wdt:P17), if reported of the trademarks

In [7]:
queryString = """
SELECT DISTINCT ?objname (COUNT( DISTINCT ?tm) AS ?number)  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P17 ?ctr.
    
?ctr <http://schema.org/name> ?objname.
?tm <http://schema.org/name> ?tmname.
} 
GROUP BY (?objname)
ORDER BY DESC (?number)
"""

print("Results")
run_query(queryString)

Results
[('objname', 'United States of America'), ('number', '4394')]
[('objname', 'United Kingdom'), ('number', '1456')]
[('objname', 'Germany'), ('number', '1046')]
[('objname', 'Japan'), ('number', '744')]
[('objname', 'France'), ('number', '508')]
[('objname', 'Italy'), ('number', '493')]
[('objname', 'Canada'), ('number', '383')]
[('objname', 'South Korea'), ('number', '282')]
[('objname', 'Australia'), ('number', '271')]
[('objname', 'Spain'), ('number', '270')]
[('objname', 'Netherlands'), ('number', '259')]
[('objname', 'Finland'), ('number', '225')]
[('objname', 'Norway'), ('number', '220')]
[('objname', 'Sweden'), ('number', '206')]
[('objname', 'Russia'), ('number', '174')]
[('objname', 'Brazil'), ('number', '157')]
[('objname', 'Poland'), ('number', '135')]
[('objname', 'Belgium'), ('number', '134')]
[('objname', 'Turkey'), ('number', '119')]
[('objname', 'Denmark'), ('number', '102')]
[('objname', 'Austria'), ('number', '77')]
[('objname', "People's Republic of China"), ('

157

#### I see that the majority of trademarks are registered in the Unites States

### Task - 5.1 How many U.S. companies hold trademarks? In which sectors?

#### Need to discover entity US

In [16]:
queryString = """
SELECT DISTINCT ?ctr ?ctrname  WHERE {

?q wdt:P17 ?ctr.
    
?ctr <http://schema.org/name> ?ctrname.
FILTER (contains(?ctrname, "United"))
} 

"""

print("Results")
run_query(queryString)

Results
[('ctr', 'http://www.wikidata.org/entity/Q3088906'), ('ctrname', 'France–United Kingdom border')]
[('ctr', 'http://www.wikidata.org/entity/Q958291'), ('ctrname', 'United Principalities')]
[('ctr', 'http://www.wikidata.org/entity/Q1061488'), ('ctrname', 'United States of Colombia')]
[('ctr', 'http://www.wikidata.org/entity/Q11285099'), ('ctrname', 'United States occupation of the Ryukyu Islands')]
[('ctr', 'http://www.wikidata.org/entity/Q170468'), ('ctrname', 'United Arab Republic')]
[('ctr', 'http://www.wikidata.org/entity/Q738144'), ('ctrname', 'United Provinces of the Río de la Plata')]
[('ctr', 'http://www.wikidata.org/entity/Q1065'), ('ctrname', 'United Nations')]
[('ctr', 'http://www.wikidata.org/entity/Q11223'), ('ctrname', 'United States Air Force')]
[('ctr', 'http://www.wikidata.org/entity/Q11703'), ('ctrname', 'United States Virgin Islands')]
[('ctr', 'http://www.wikidata.org/entity/Q145'), ('ctrname', 'United Kingdom')]
[('ctr', 'http://www.wikidata.org/entity/Q15864

36

#### Found United States of America (wd:Q30)

#### Search for US companies that hold trademark

In [5]:
queryString = """
SELECT DISTINCT ?comname  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?com.
?com wdt:P17 wd:Q30.
    
?com <http://schema.org/name> ?comname.
} 
"""
print("Results")
run_query(queryString)

Results
[('comname', 'The Vanguard Group')]
[('comname', 'Boston Partners')]
[('comname', 'Sasco Capital')]
[('comname', 'Wikimedia Foundation')]
[('comname', 'WarnerMedia')]
[('comname', 'AT&T')]
[('comname', 'National Amusements')]
[('comname', 'GiFi')]
[('comname', 'Silver Lake Partners')]
[('comname', 'Anschutz Entertainment Group')]
[('comname', 'Cinemark Theatres')]
[('comname', 'Anheuser-Busch')]
[('comname', 'Pabst Brewing Company')]
[('comname', 'Warner Music Group')]
[('comname', 'BlackRock')]
[('comname', 'Universal Music Group')]
[('comname', 'Sony Music')]
[('comname', 'Kohlberg Kravis Roberts')]
[('comname', 'State Street Corporation')]
[('comname', 'Barnes & Noble')]
[('comname', 'Tru Kids')]
[('comname', 'Sun Capital Partners')]
[('comname', 'Starbucks')]
[('comname', 'Hilco')]
[('comname', 'Verizon')]
[('comname', 'Amazon')]
[('comname', 'Kroger')]
[('comname', 'The Pantry')]
[('comname', 'Leonard Green & Partners')]
[('comname', 'American Greetings')]
[('comname', 'Ic

311

#### I count them

In [12]:
queryString = """
SELECT DISTINCT (COUNT(DISTINCT ?com) as ?number) WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?com.
?com wdt:P17 wd:Q30.
} 
"""
print("Results")
run_query(queryString)

Results
[('number', '311')]


1

#### ...and i aggregate them by the type

In [14]:
queryString = """
SELECT DISTINCT ?typename (COUNT(DISTINCT ?com) as ?number) WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P127 ?com.
?com wdt:P17 wd:Q30;
     wdt:P452 ?type.
    
?type <http://schema.org/name> ?typename.
} 
GROUP BY (?typename)
ORDER BY DESC(?number)
"""
print("Results")
run_query(queryString)

Results
[('typename', 'retail'), ('number', '24')]
[('typename', 'food industry'), ('number', '18')]
[('typename', 'fast-moving consumer goods'), ('number', '9')]
[('typename', 'phonographic industry'), ('number', '9')]
[('typename', 'telecommunications industry'), ('number', '8')]
[('typename', 'music industry'), ('number', '8')]
[('typename', 'financial services'), ('number', '7')]
[('typename', 'clothing industry'), ('number', '6')]
[('typename', 'hospitality industry'), ('number', '6')]
[('typename', 'publishing'), ('number', '6')]
[('typename', 'automotive industry'), ('number', '6')]
[('typename', 'food processing'), ('number', '6')]
[('typename', 'beverage industry'), ('number', '5')]
[('typename', 'television'), ('number', '5')]
[('typename', 'petroleum industry'), ('number', '5')]
[('typename', 'media industry'), ('number', '5')]
[('typename', 'manufacturing'), ('number', '4')]
[('typename', 'electronics industry'), ('number', '4')]
[('typename', 'software industry'), ('number

127

## Task - 5.2 How many people hold or are connected to trademarks? In which role?

#### I already found the founders of the companies that hold the trademarks in task , now i search the properties "chief executive office" (wdt:P169) and "director/manager" (wdt:P1037)

In [23]:
queryString = """
SELECT DISTINCT ?tmname ?objname WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P169 ?obj.

?tm <http://schema.org/name> ?tmname.
?obj <http://schema.org/name> ?objname.
} 

"""

print("Results")
run_query(queryString)

Results
[('tmname', 'FNC Entertainment'), ('objname', 'Han Sung Ho')]
[('tmname', 'YG Entertainment'), ('objname', 'Yang Hyun-seok')]
[('tmname', 'YG Entertainment'), ('objname', 'Yang Min-suk')]
[('tmname', 'AMC Theatres'), ('objname', 'Adam Aron')]
[('tmname', 'William Morris Endeavor'), ('objname', 'Ari Emanuel')]
[('tmname', 'William Morris Endeavor'), ('objname', 'Patrick Whitesell')]
[('tmname', 'National Amusements'), ('objname', 'Sumner Redstone')]
[('tmname', "Macy's"), ('objname', 'Terry J. Lundgren')]
[('tmname', 'Amuse'), ('objname', 'Masaki Nakanishi')]
[('tmname', 'AOMG'), ('objname', 'Simon Dominic')]
[('tmname', 'AOMG'), ('objname', 'Jay Park')]
[('tmname', 'La Maison Simons'), ('objname', 'Peter Simons')]
[('tmname', 'Multikino'), ('objname', 'Piotr Zygo')]
[('tmname', 'People Store'), ('objname', 'Rebecca Shrager')]
[('tmname', 'EMKA Productions'), ('objname', "Steve O'Rourke")]
[('tmname', 'Bryan Morrison Agency'), ('objname', 'Bryan Morrison')]
[('tmname', 'Best Buy

149

In [25]:
queryString = """
SELECT DISTINCT ?tmname ?objname WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
    wdt:P1037 ?obj.

?tm <http://schema.org/name> ?tmname.
?obj <http://schema.org/name> ?objname.
} 

"""

print("Results")
run_query(queryString)

Results
[('tmname', 'Pathé'), ('objname', 'Jérôme Seydoux')]
[('tmname', 'UGC'), ('objname', 'Guy Verrecchia')]
[('tmname', 'JYP Entertainment'), ('objname', 'J.Y. Park')]
[('tmname', 'La Maison Simons'), ('objname', 'Peter Simons')]
[('tmname', 'Kinostar'), ('objname', 'Michael Roesch')]
[('tmname', 'Walmart'), ('objname', 'Doug McMillon')]
[('tmname', 'Motion Music Group'), ('objname', 'Gabriel Chuquimia')]
[('tmname', 'Cartier'), ('objname', 'Bernard Fornas')]
[('tmname', 'Microsoft Store'), ('objname', 'Satya Nadella')]
[('tmname', 'Carrefour'), ('objname', 'Alexandre Bompard')]
[('tmname', 'H&M'), ('objname', 'Karl-Johan Persson')]
[('tmname', 'Globus'), ('objname', 'Thomas Bruch')]
[('tmname', 'Reserved'), ('objname', 'Marek Piechocki')]
[('tmname', 'The Great Atlantic & Pacific Tea Company'), ('objname', 'Christian W.E. Haub')]
[('tmname', 'Magnit'), ('objname', 'Sergey Galitsky')]
[('tmname', 'Promovacances'), ('objname', 'Alain de Mendonça')]
[('tmname', 'Crédit Lyonnais'), ('

82

## Task 5.3

In [3]:
queryString = """
SELECT DISTINCT  ?typename (COUNT(?tm) AS ?number)  WHERE {

?tm wdt:P31/wdt:P279* wd:Q167270;
        wdt:P452 ?type.

?type <http://schema.org/name> ?typename.
} 
GROUP BY (?typename)
ORDER BY DESC (?number)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('typename', 'retail'), ('number', '1652')]


1